### 10/08言われたこと（来週までの課題）

- 以下両方とも伊庭さんの資料を参照せよ
- $β$を使う式を使用せよ
- レプリカ交換モンテカルロ法を実装せよ（温度を複数設定する）⇒　よりバラエティに飛んだサンプルを得ることができるようになるだろう

<br/>

#### ヒントとなるアイデア
- ニューラルネットは重回帰分析を行う手法の1つとして見ることができます。
- (北井さん)ニューラルネットやガウス過程等いろいろ試してみるとよいです。どの手法にも言えることですが過学習してスコアが出ないということがあるので、訓練誤差も見ながら正則化項を調整してみるとよいです。

#### 10/12 yとして使う値を0行目ではないものにしてみる
(理由)線形回帰モデルの精度がほぼ0だったから

１．最大値を使ってみる

In [1]:
# 説明変数
import pandas as pd
data_SNP = pd.read_table('CIMMYTwheat_markers.txt', header=None,  sep =  ' ')
print('(SNPサンプル数, 塩基数)=', data_SNP.shape)
#print(data_SNP.style)
#print('SNPサンプル数：', len(data_SNP))
#print(data_SNP.head)

(SNPサンプル数, 塩基数)= (599, 1279)


In [2]:
# 目的変数
data_yields = pd.read_table('CIMMYTwheat_yields.txt', header=None,  sep =  ' ')
print('SNPサンプル数：', len(data_yields))
print(type(data_yields))
print(data_yields.shape)
print(data_yields.iloc[0]) #0行目
print(data_yields.iloc[0, 0]) #[0, 0]


data_amount_list = []
for i in range(len(data_yields)):
    canditate = data_yields.iloc[i, 0]
    for j in range(1, 4):
        if data_yields.iloc[i, j] >= canditate:
            canditate = data_yields.iloc[i, j]
    data_amount_list.append(canditate)


#print(data_amount)
print(len(data_amount_list))
print(type(data_amount_list))
print(data_amount_list[3])

SNPサンプル数： 599
<class 'pandas.core.frame.DataFrame'>
(599, 4)
0    1.671629
1   -1.727470
2   -1.890285
3    0.050916
Name: 0, dtype: float64
1.671629
599
<class 'list'>
0.7854395000000001


In [4]:
data_amount = pd.Series(data_amount_list)

In [5]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data_SNP, data_amount, test_size=0.3) #x, y ,test0.3, train0.7

### 特徴量選択
- 入力データが高次元のとき，データからの学習が非常に困難になることが知られている（“次元の呪い”）． 入力データの次元をうまく減らすことができれば，その後の学習が行ないやすくなる． 
- 次元削減の目的は，本質的な情報を失うことなくデータの次元を削減することである． 
- もとの特徴の部分集合を低次元表現として取り出すとき，次元削減は特徴選択とよばれる．生物学や化学などの科学的な応用場面では予測能力よりも学習結果の説明能力が重視されるため，特徴選択が好まれる．
- もとの特徴の（非線形）結合として低次元表現として取り出すとき， 次元選択は特徴抽出と呼ばれる．パターン認識などの工学的な応用場面では， 予測能力が重要なため特徴抽出の方が有用である．


#### 教師付き次元削減
教師付き次元削減では，出力を最も良く説明する入力変数の部分集合，および，入力空間の部分空間を見つけることが目的である．

### ベイズ線形回帰 

ベイズ線形回帰は、データ分布から関数を導き出す際にベイズ確率を使う教師あり学習。信頼度の高そうなデータと信頼度の低そうなデータ（ノイズっぽい）を同等に扱わず、重みを変えるのです。 ベイズ線形回帰はアプローチは違えどノイズを過小評価して過学習を防ぐ正則化と同じ結果になる。 
- L1ノルム正則化(Lasso回帰)：極端なデータの重みを0にする 
- L2ノルム正則化(Ridge回帰)：極端なデータの重みを0に近づける 



In [13]:
# Lasso回帰
import numpy as np
from sklearn.linear_model import Lasso
lasso = Lasso().fit(x_train, y_train)
print("Training set score: {:.2f}".format(lasso.score(x_train, y_train)))
print("Test set score: {:.2f}".format(lasso.score(x_test, y_test)))
print("Number of features used:", np.sum(lasso.coef_ != 0))

Training set score: 0.00
Test set score: -0.04
Number of features used: 0


In [6]:
# モデルベース特徴量選択（embedded method、組み込み法）
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor

# estimator として RandomForestRegressor を使用。重要度が median 以上のものを選択
select = SelectFromModel(RandomForestRegressor(n_estimators = 100, random_state = 42), threshold = 'median')
select.fit(x_train, y_train)

#yはselect.fitしなくていい
x_train_selected = select.transform(x_train)
x_test_selected = select.transform(x_test)

print('x_train.shape: {}'.format(x_train.shape))
print('x_train_l1_shape: {}'.format(x_train_selected.shape))

mask = select.get_support()
print(mask)
print(type(mask))

x_train.shape: (419, 1279)
x_train_l1_shape: (419, 640)
[ True  True  True ...  True  True False]
<class 'numpy.ndarray'>


In [7]:
mask = select.get_support()
print(mask)
print(type(mask))
print(len(mask)) #もとの特徴量の数

[ True  True  True ...  True  True False]
<class 'numpy.ndarray'>
1279


In [8]:
from sklearn import preprocessing, utils

lab_enc = preprocessing.LabelEncoder()

train_scores_encoded = lab_enc.fit_transform(y_train)
print(utils.multiclass.type_of_target(train_scores_encoded))

test_scores_encoded = lab_enc.fit_transform(y_test)
print(utils.multiclass.type_of_target(test_scores_encoded))

multiclass
multiclass


In [9]:
#線形回帰（最小二乗法）
from sklearn.linear_model import LinearRegression
lr = LinearRegression().fit(x_train_selected, y_train)
print("Training set score: {:.2f}".format(lr.score(x_train_selected, y_train)))
print("Test set score: {:.2f}".format(lr.score(x_test_selected, y_test)))
# 過剰適合

Training set score: 1.00
Test set score: -1.82


In [10]:
# リッジ回帰(デフォルトで、alpha=1.0)
from sklearn.linear_model import Ridge
ridge = Ridge().fit(x_train_selected, y_train)
print("Training set score: {:.2f}".format(ridge.score(x_train_selected, y_train)))
print("Test set score: {:.2f}".format(ridge.score(x_test_selected, y_test)))
# まだ過剰適合

# 係数とは w のこと
# Ridgeモデルでは、モデルの簡潔さ（絶対値が0に近い係数の数）と訓練セットに対する性能がトレードオフ
# alphaを増やすと、係数はより0に近くなり、訓練セットに対する性能は低下するが、汎化にはそのほうが良い可能性もある

Training set score: 0.99
Test set score: -0.93


In [11]:
# リッジ回帰(alpha=10)
from sklearn.linear_model import Ridge
ridge = Ridge(alpha = 10).fit(x_train_selected, y_train)
print("Training set score: {:.2f}".format(ridge.score(x_train_selected, y_train)))
print("Test set score: {:.2f}".format(ridge.score(x_test_selected, y_test)))
# alpha=10にしたにもかかわらずテストスコアが低すぎる

Training set score: 0.87
Test set score: -0.17


In [12]:
# リッジ回帰
from sklearn.linear_model import Ridge
ridge = Ridge(alpha = 20).fit(x_train_selected, y_train)
print("Training set score: {:.2f}".format(ridge.score(x_train_selected, y_train)))
print("Test set score: {:.2f}".format(ridge.score(x_test_selected, y_test)))
# まだ過剰適合

Training set score: 0.80
Test set score: -0.02


(学習に関してわかったこと)
-  最大値を使ったが、それでもだめだった
- （特徴量選択＋）線形回帰では表せない（教師あり学習、線形回帰）
- lasso回帰でしてもスコアでない

２．最小値をつかってみる

In [14]:
# 説明変数
import pandas as pd
data_SNP = pd.read_table('CIMMYTwheat_markers.txt', header=None,  sep =  ' ')
print('(SNPサンプル数, 塩基数)=', data_SNP.shape)
#print(data_SNP.style)
#print('SNPサンプル数：', len(data_SNP))
#print(data_SNP.head)

(SNPサンプル数, 塩基数)= (599, 1279)


In [15]:
# 目的変数
data_yields = pd.read_table('CIMMYTwheat_yields.txt', header=None,  sep =  ' ')
print('SNPサンプル数：', len(data_yields))
print(type(data_yields))
print(data_yields.shape)
print(data_yields.iloc[0]) #0行目
print(data_yields.iloc[0, 0]) #[0, 0]


data_amount_list = []
for i in range(len(data_yields)):
    canditate = data_yields.iloc[i, 0]
    for j in range(1, 4):
        if data_yields.iloc[i, j] <= canditate:
            canditate = data_yields.iloc[i, j]
    data_amount_list.append(canditate)


#print(data_amount)
print(len(data_amount_list))
print(type(data_amount_list))
print(data_amount_list[3])

SNPサンプル数： 599
<class 'pandas.core.frame.DataFrame'>
(599, 4)
0    1.671629
1   -1.727470
2   -1.890285
3    0.050916
Name: 0, dtype: float64
1.671629
599
<class 'list'>
0.0939492


In [16]:
data_amount = pd.Series(data_amount_list)

In [17]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data_SNP, data_amount, test_size=0.3) #x, y ,test0.3, train0.7

In [24]:
# Lasso回帰
import numpy as np
from sklearn.linear_model import Lasso
lasso = Lasso().fit(x_train, y_train)
print("Training set score: {:.2f}".format(lasso.score(x_train, y_train)))
print("Test set score: {:.2f}".format(lasso.score(x_test, y_test)))
print("Number of features used:", np.sum(lasso.coef_ != 0))

Training set score: 0.00
Test set score: -0.03
Number of features used: 0


In [18]:
# モデルベース特徴量選択（embedded method、組み込み法）
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor

# estimator として RandomForestRegressor を使用。重要度が median 以上のものを選択
select = SelectFromModel(RandomForestRegressor(n_estimators = 100, random_state = 42), threshold = 'median')
select.fit(x_train, y_train)

#yはselect.fitしなくていい
x_train_selected = select.transform(x_train)
x_test_selected = select.transform(x_test)

print('x_train.shape: {}'.format(x_train.shape))
print('x_train_l1_shape: {}'.format(x_train_selected.shape))

mask = select.get_support()
print(mask)
print(type(mask))

x_train.shape: (419, 1279)
x_train_l1_shape: (419, 640)
[ True  True False ...  True  True  True]
<class 'numpy.ndarray'>


In [19]:
mask = select.get_support()
print(mask)
print(type(mask))
print(len(mask)) #もとの特徴量の数

[ True  True False ...  True  True  True]
<class 'numpy.ndarray'>
1279


In [20]:
from sklearn import preprocessing, utils

lab_enc = preprocessing.LabelEncoder()

train_scores_encoded = lab_enc.fit_transform(y_train)
print(utils.multiclass.type_of_target(train_scores_encoded))

test_scores_encoded = lab_enc.fit_transform(y_test)
print(utils.multiclass.type_of_target(test_scores_encoded))

multiclass
multiclass


In [21]:
#線形回帰（最小二乗法）
from sklearn.linear_model import LinearRegression
lr = LinearRegression().fit(x_train_selected, y_train)
print("Training set score: {:.2f}".format(lr.score(x_train_selected, y_train)))
print("Test set score: {:.2f}".format(lr.score(x_test_selected, y_test)))
# 過剰適合

Training set score: 1.00
Test set score: -2.43


In [22]:
# リッジ回帰(デフォルトで、alpha=1.0)
from sklearn.linear_model import Ridge
ridge = Ridge().fit(x_train_selected, y_train)
print("Training set score: {:.2f}".format(ridge.score(x_train_selected, y_train)))
print("Test set score: {:.2f}".format(ridge.score(x_test_selected, y_test)))
# まだ過剰適合

# 係数とは w のこと
# Ridgeモデルでは、モデルの簡潔さ（絶対値が0に近い係数の数）と訓練セットに対する性能がトレードオフ
# alphaを増やすと、係数はより0に近くなり、訓練セットに対する性能は低下するが、汎化にはそのほうが良い可能性もある

Training set score: 0.98
Test set score: -1.17


In [23]:
# リッジ回帰(alpha=10)
from sklearn.linear_model import Ridge
ridge = Ridge(alpha = 10).fit(x_train_selected, y_train)
print("Training set score: {:.2f}".format(ridge.score(x_train_selected, y_train)))
print("Test set score: {:.2f}".format(ridge.score(x_test_selected, y_test)))
# alpha=10にしたにもかかわらずテストスコアが低すぎる

Training set score: 0.88
Test set score: -0.37


In [26]:
# リッジ回帰(alpha=100)
from sklearn.linear_model import Ridge
ridge = Ridge(alpha = 100).fit(x_train_selected, y_train)
print("Training set score: {:.2f}".format(ridge.score(x_train_selected, y_train)))
print("Test set score: {:.2f}".format(ridge.score(x_test_selected, y_test)))
# alpha=10にしたにもかかわらずテストスコアが低すぎる

Training set score: 0.61
Test set score: 0.05


(学習に関してわかったこと)
-  最小値を使ったが、それでもだめだった
- （特徴量選択＋）線形回帰では表せない（教師あり学習、線形回帰）
- lasso回帰でしてもスコアでない
-  線形回帰を諦めたほうがよさそう

３．ニューラルネットワーク

- https://qiita.com/sugiyamath/items/8470ec843f2d213576f1

In [1]:
# 隠れ層をfitメソッドで指定できるように作りました。
# 重みの更新はtrain_stepで指定したOptimizerによるコスト関数最小化で行ってくれるので、
# バックプロパゲーションを手動で構築する必要はありません。
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

class nnet:

    def __init__(self, X, random_state=3):
        self.sess = tf.Session()
        seed = random_state
        tf.set_random_seed(seed)
        np.random.seed(seed)
        self.x_data = tf.placeholder(shape=[None, X.shape[1]], dtype=tf.float32)
        self.y_target = tf.placeholder(shape=[None, 1], dtype=tf.float32)

    def init_weight(self, shape, st_dev):
        return tf.Variable(tf.random_normal(shape, stddev=st_dev))

    def init_bias(self, shape, st_dev):
        return tf.Variable(tf.random_normal(shape, stddev=st_dev))

    def fully_connected(self, input_layer, weights, biases):
        return tf.nn.relu(tf.add(tf.matmul(input_layer, weights), biases))

    def fit(self, X, y, hidden_size, batch_size=100, iter_size=200):

        x_data = self.x_data
        y_target = self.y_target

        final_output = self.build_hidden_layer(hidden_size, X.shape[1])

        self.loss = tf.reduce_mean(tf.abs(y_target - final_output))
        self.opt = tf.train.AdamOptimizer(0.05)
        self.train_step = self.opt.minimize(self.loss)

        init = tf.global_variables_initializer()
        self.sess.run(init)

        loss_vec = self.train(X, y, iter_size, batch_size)
        plt.plot(loss_vec)
        return final_output

    def predict(self, final_output, X):
        x_data = self.x_data
        return [val[0] for val in self.sess.run(final_output, feed_dict={x_data: X})]


    def train(self, X, y, iter_size, batch_size):
        loss_vec = []
        x_data = self.x_data
        y_target = self.y_target
        for i in range(iter_size):
            rand_index = np.random.choice(len(X), size=batch_size)
            rand_x = X[rand_index]
            rand_y = np.transpose([y[rand_index]])
            self.sess.run(self.train_step, feed_dict={x_data:rand_x, y_target: rand_y})
            loss_vec.append(self.sess.run(self.loss, feed_dict={x_data:rand_x, y_target: rand_y}))

            if (i+1)%25==0:
                print('Generation:'+str(i+1)+', Loss = '+str(loss_vec[-1]))

        return loss_vec


    def build_hidden_layer(self, hidden_size, col_size):
        weights = []
        biases = []
        layers = []
        tmp_size = col_size
        x_data = self.x_data
        last_layer = x_data

        for hsize in hidden_size:
            weights.append(self.init_weight(shape=[tmp_size, hsize], st_dev=10.0))
            biases.append(self.init_bias(shape=[hsize], st_dev=10.0))
            layers.append(self.fully_connected(last_layer, weights[-1], biases[-1]))
            tmp_size = hsize
            last_layer = layers[-1]

        weights.append(self.init_weight(shape=[tmp_size, 1], st_dev=10.0))
        biases.append(self.init_bias(shape=[1], st_dev=10.0))
        layers.append(self.fully_connected(last_layer, weights[-1], biases[-1]))
        final_output = layers[-1]

        return final_output

/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/Caskroom/miniconda/base/lib/python3

In [3]:
# 説明変数
import pandas as pd
data_SNP = pd.read_table('CIMMYTwheat_markers.txt', header=None,  sep =  ' ')
print('(SNPサンプル数, 塩基数)=', data_SNP.shape)

(SNPサンプル数, 塩基数)= (599, 1279)


In [4]:
# 目的変数
data_yields = pd.read_table('CIMMYTwheat_yields.txt', header=None,  sep =  ' ')
print('SNPサンプル数：', len(data_yields))
print(type(data_yields))
print(data_yields.shape)
print(data_yields.iloc[0]) #0行目
print(data_yields.iloc[0, 0]) #[0, 0]

# yを0列目にした（戻した）
data_amount_list = []
for i in range(len(data_yields)):
    data_amount_list.append(data_yields.iloc[i, 0])

#print(data_amount)
print(len(data_amount_list))
print(type(data_amount_list))
print(data_amount_list[3])

SNPサンプル数： 599
<class 'pandas.core.frame.DataFrame'>
(599, 4)
0    1.671629
1   -1.727470
2   -1.890285
3    0.050916
Name: 0, dtype: float64
1.671629
599
<class 'list'>
0.7854395000000001


In [5]:
data_amount = pd.Series(data_amount_list)

In [6]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data_SNP, data_amount, test_size=0.3) #x, y ,test0.3, train0.7

In [8]:
%matplotlib inline
nnetc = nnet(x_train)
model = nnetc.fit(x_train, y_train.ravel(), hidden_size=[25, 10, 3], iter_size=200)

ValueError: Cannot feed value of shape (419, 100) for Tensor 'Placeholder:0', which has shape '(?, 1279)'